# Context
This notebook is essentially copy-paste from notebooks developed bby Mathias Régnier and Simon Biquard   

In [ ]:
import pysm3
import pysm3.units as u
import numpy as np
import numpy.ma as ma
import healpy as hp
import pickle
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import pickle
from astropy.io import fits

from qubic import camb_interface as qc
import fgbuster as fgb

import warnings
warnings.filterwarnings("ignore")

import qubic
center = qubic.equ2gal(0, -57)

rc('figure', figsize=(16, 10))
rc('font', size=15)


In [ ]:
### Some usefull functions (see FGB-FullSky-NoNoise.Rmd)
def separate(comp, instr, maps_to_separate, tol=1e-12, print_option=True):
    solver_options = {}
    solver_options['disp'] = True
    fg_args = comp, instr, maps_to_separate
    fg_kwargs = {'method': 'BFGS', 'tol': 1e-12, 'options': solver_options}
    try:
        res = fgb.basic_comp_sep(*fg_args, **fg_kwargs)
    except KeyError:
        fg_kwargs['options']['disp'] = False
        res = fgb.basic_comp_sep(*fg_args, **fg_kwargs)
    if print_option:
        print()
        print("message:", res.message)
        print("success:", res.success)
        print("result:", res.x)
    return res


def reconvolve(maps, fwhms, ref_fwhm, verbose=False):
    if verbose: 
        print('Reconvolution to coommon FWHM')
    sig_conv = np.sqrt(ref_fwhm**2 - fwhms**2)
    maps_out = np.zeros_like(maps)
    for i in range(len(fwhms)):
        if sig_conv[i] == 0:
            if verbose:
                print('Map {0:} fwhmin={1:6.3f} fwhmout={2:6.3f} => We do not reconvolve'.format(i, 
                                                                                             fwhms[i], ref_fwhm))
            maps_out[i,:] = maps[i,:]
        else:
            if verbose:
                print('Map {0:} fwhmin={1:6.3f} fwhmout={2:6.3f} => We reconvolve with {3:6.3f}'.format(i, 
                                                                                                    fwhms[i], 
                                                                                                    ref_fwhm, 
                                                                                                    sig_conv[i]))
            maps_out[i,:] = hp.smoothing(maps[i,:], fwhm=np.deg2rad(sig_conv[i]), pol=True, verbose=False)
    return maps_out

# this function is from Simon Biquard
def get_alm_maps(pixel_maps, fwhms, resol_correction=False, ref_arcmin=0, pixwin_correction=False, verbose=False):
    """
    Compute alm maps from pixel maps and format them for FgBuster.
    """
    sh = np.shape(pixel_maps)
    nside = hp.npix2nside(sh[2])
    n = sh[0]
    lmax = 2*nside+1
    ell = np.arange(start=0, stop= lmax+1)

    ref_sigma_rad = np.deg2rad(ref_arcmin / 60.) / 2.355
    ref_fl = np.exp(- 0.5 * np.square(ref_sigma_rad * ell))

    if verbose: 
        print('In get_alm_maps: FWHM = ', fwhms)
    beam_sigmas_rad = np.deg2rad(fwhms) / (2*np.sqrt(2*np.log(2)))
    pixwin = hp.pixwin(nside, lmax=lmax) if pixwin_correction else np.ones(lmax + 1)

    # compute maps
    alm_maps = None
    for f in range(n):
        alms = hp.map2alm(pixel_maps[f], lmax=lmax, pol=True)
        correction = None
        if f == 0:
            sh = np.shape(alms)
            alm_maps = np.empty((n, sh[0], 2 * sh[1]))
        if resol_correction:
            print('Applying Resol Correction')
            gauss_fl = np.exp(- 0.5 * np.square(beam_sigmas_rad[f] * ell))
            correction = ref_fl / gauss_fl / pixwin
        else:
            print('No Resol Correction applied')
        for i, t in enumerate(alms):
            alm_maps[f, i] = format_alms(hp.almxfl(t, correction) if resol_correction else t)
    return alm_maps

# credits to J. Errard for these two functions
def intersect_mask(maps):
    if hp.pixelfunc.is_ma(maps):
        mask = maps.mask
    else:
        mask = maps == hp.UNSEEN

    # Mask entire pixel if any of the frequencies in the pixel is masked
    return np.any(mask, axis=tuple(range(maps.ndim - 1)))

def format_alms(alms, lmin=0, nulling_option=True):
    lmax = hp.Alm.getlmax(alms.shape[-1])
    alms = np.asarray(alms, order='C')
    alms = alms.view(np.float64)
    em = hp.Alm.getlm(lmax)[1]
    em = np.stack((em, em), axis=-1).reshape(-1)
    mask_em = [m != 0 for m in em]
    alms[..., mask_em] *= np.sqrt(2)
    if nulling_option:
        alms[..., np.arange(1, lmax + 1, 2)] = hp.UNSEEN  # mask imaginary m = 0
        mask_alms = intersect_mask(alms)
        alms[..., mask_alms] = 0  # thus no contribution to the spectral likelihood
    alms = np.swapaxes(alms, 0, -1)
    if lmin != 0:
        ell = hp.Alm.getlm(lmax)[0]
        ell = np.stack((ell, ell), axis=-1).reshape(-1)
        mask_lmin = [ll < lmin for ll in ell]
        if nulling_option:
            alms[mask_lmin, ...] = hp.UNSEEN
    return alms


In [ ]:
#### General Stuff ####
nside = 256
stk = ['I', 'Q', 'U']

### PySM stuff
we generate PySM maps with CMB + Dust

In [ ]:
#### PYSM Stuff ####
# generate I,Q,U sky maps with PySM
unit = u.uK_CMB
np.random.seed(42)
sky = pysm3.Sky(nside=nside, preset_strings=['d0', 'c1'], output_unit=unit)
sky_cmb = pysm3.Sky(nside=nside, preset_strings=['c1'], output_unit=unit)
sky_dust = pysm3.Sky(nside=nside, preset_strings=['d0'], output_unit=unit)


Now we generate some frequency maps

In [ ]:
freqs = np.array([200, 220, 240])

# CMB+dust
sky_emissions = [sky.get_emission(f * u.GHz) for f in freqs]
maps = np.array([emission.value for emission in sky_emissions])

# CMB
sky_emissions_cmb = sky_cmb.get_emission(150 * u.GHz)
maps_cmb = np.array([emission.value for emission in sky_emissions_cmb])

sky_emissions_dust = sky_dust.get_emission(150 * u.GHz)
maps_dust = np.array([emission.value for emission in sky_emissions_dust])

print(np.shape(maps))   #nu, Stk, pix
print(np.shape(maps_cmb))   #nu, Stk, pix
print(np.shape(maps_dust))   #nu, Stk, pix

rc('figure', figsize=(16, 10))
suptitle('Simulated maps for each frequency', fontsize=30,y=1.05)
nf = maps.shape[0]
nstk = maps.shape[1]
print(maps.shape)
for i in range(nf):
    for j in range(nstk):
        ss = np.std(maps[0,j,:])
        hp.mollview(maps[i,j,:], sub=(nf,3,3*i+j+1), min = -3*ss, max=3*ss,
                   title='CMB+Dust {} nu={} GHz'.format(stk[j], freqs[i]))
tight_layout()

In [ ]:
rc('figure', figsize=(16, 4))
suptitle('Input Dust only at 150 GHz', fontsize=30)
for j in range(nstk):
    ss = np.std(maps_dust[j,:])
    hp.mollview(maps_dust[j,:], sub=(1,3,j+1), min = -3*ss, max=3*ss,
               title='Dust {} nu=150 GHz'.format(stk[j], freqs[i]))
tight_layout()

figure()
suptitle('Input CMB only at 150 GHz', fontsize=30)
for j in range(nstk):
    ss = np.std(maps_cmb[j,:])
    hp.mollview(maps_cmb[j,:], sub=(1,3,j+1), min = -3*ss, max=3*ss,
               title='CMB {} nu=150 GHz'.format(stk[j]))
tight_layout()


### Cut-Sky: from QUBIC
Let's use the QUBIC cut-sky, as this is noiseless, we do not bother about the coverage values themselves, we just keep all pixels above some threshold on the coverage.

In [ ]:
covmap = hp.ud_grade(hp.read_map('QUBIC-Typical-Coverage.fits'), nside)

suptitle('Cut-Sky (from QUBIC)', fontsize=30, y=1.2)
hp.mollview(covmap, title='QUBIC Cov Map', sub=(1,2,1))

thr = 0.1
mymask = (covmap > (np.max(covmap)*thr)).astype(int)
hp.mollview(mymask, title='Mask with thr={}'.format(thr), sub=(1,2,2))

pixok = mymask > 0

# Cut-Sky simulations with identical resolutions => should work
We now create noiseless reconstructed cut-sky maps at various frequencies, but with the simplification that they are at the same angular resolution.

In [ ]:
myFWHM = 0.5 # degrees
maps_conv = np.array([hp.smoothing(m.copy(), fwhm=np.deg2rad(myFWHM), pol=True, verbose=False) for m in maps])
maps_cmb_conv = hp.smoothing(maps_cmb, fwhm=np.deg2rad(myFWHM), pol=True, verbose=False)
maps_dust_conv = hp.smoothing(maps_dust, fwhm=np.deg2rad(myFWHM), pol=True, verbose=False)
for j in range(3):
    maps_cmb_conv[j,~pixok] = hp.UNSEEN
    maps_dust_conv[j,~pixok] = hp.UNSEEN
    for i in range(nf):
        maps_conv[i,j,~pixok] = hp.UNSEEN        

print(maps_cmb[1,pixok])
print(maps_cmb_conv[1,pixok])

In [ ]:
rc('figure', figsize=(16, 10))
suptitle('Cut-sky maps per freq. at same resolution', fontsize=30, y=1.05)
nf = maps.shape[0]
nstk = maps.shape[1]
print(maps.shape)
for i in range(nf):
    for j in range(nstk):
        ss = np.std(maps_conv[0,j,pixok])
        hp.gnomview(maps_conv[i,j,:], sub=(nf,3,3*i+j+1), min = -3*ss, max=3*ss,
                   title='CMB+Dust {} nu={} GHz'.format(stk[j], freqs[i]), rot=center, reso=15)
tight_layout()

In [ ]:
rc('figure', figsize=(16, 5))

figure()
suptitle('Dust Only input', fontsize=30, y=1.1)
for j in range(nstk):
    ss = np.std(maps_dust_conv[j,pixok])
    hp.gnomview(maps_dust_conv[j,:], sub=(1,3,j+1), min = -3*ss, max=3*ss,
               title='Dust {} nu=150 GHz'.format(stk[j]), rot=center, reso=15)
tight_layout()

figure()
suptitle('CMB Only input', fontsize=30, y=1.1)
for j in range(nstk):
    ss = np.std(maps_cmb_conv[j,pixok])
    hp.gnomview(maps_cmb_conv[j,:], sub=(1,3,j+1), min = -3*ss, max=3*ss,
               title='CMB {} nu=150 GHz'.format(stk[j]), rot=center, reso=15)
tight_layout()


In [ ]:
### FGB Configuration
instrument = fgb.get_instrument('Qubic')
instrument.frequency = freqs
instrument.fwhm = np.zeros(len(freqs)) + myFWHM
components = [fgb.Dust(150., temp=20.), fgb.CMB()]

r = separate(components, instrument, maps_conv)
print('Resulting beta: {}'.format(r.x[0]))

In [ ]:
print(np.sum(pixok))
### Resulting CMB and Dust
rc('figure', figsize=(16, 12))
dustrec = r.s[0,:,:]
suptitle('Reconstructed dust (pixel based): Same Resolution', fontsize=30, y=1.05)
for i in range(3):
    hp.gnomview(dustrec[i,:], sub=(2,3,i+1), title='Dust Rec: {}'.format(stk[i]), 
                min=-3*np.std(dustrec[i,pixok]), max=3*np.std(dustrec[i,pixok]),
               rot=center, reso=15)
    hp.gnomview(dustrec[i,:]-maps_dust_conv[i,:], sub=(2,3,3+i+1), 
                title='Dust Residuals: {0:} \n RMS={1:4.2g}'.format(stk[i], 
                                                                    np.std(dustrec[i,pixok]-maps_dust_conv[i,pixok])), 
                min=-3*np.std(dustrec[i,pixok]), max=3*np.std(dustrec[i,pixok]),
               rot=center, reso=15)
    if i==1:
        print(dustrec[i,pixok])
        print(maps_dust_conv[i,pixok])
        print(dustrec[i,pixok]-maps_dust_conv[i,pixok])

tight_layout()

In [ ]:
rc('figure', figsize=(16, 12))
cmbrec = r.s[1,:,:]
suptitle('Reconstructed CMB (pixel based): Same Resolution', fontsize=30, y=1.05)
for i in range(3):
    hp.gnomview(cmbrec[i,:], sub=(2,3,i+1), title='CMB Rec: {}'.format(stk[i]), 
               min=-3*np.std(cmbrec[i,pixok]), max=3*np.std(cmbrec[i,pixok]),
               rot=center, reso=15)
    hp.gnomview(cmbrec[i,:]-maps_cmb_conv[i,:], sub=(2,3,3+i+1), 
                title='CMB Residuals: {0:} \n RMS={1:4.2g}'.format(stk[i], np.std(cmbrec[i,pixok]-maps_cmb_conv[i,pixok])), 
               min=-3*np.std(cmbrec[i,pixok]), max=3*np.std(cmbrec[i,pixok]),
               rot=center, reso=15)
    if i==1:
        print(cmbrec[i,pixok])
        print(maps_cmb_conv[i,pixok])
        print(cmbrec[i,pixok]-maps_cmb_conv[i,pixok])



In [ ]:
suptitle('Residuals (pixel based): Same Resolution', fontsize=30, y=1.05)
for i in range(3):
    subplot(3,1,i+1)
    resid_cmb = cmbrec[i,pixok]-maps_cmb_conv[i,pixok]
    hist(resid_cmb, range=[-5*np.std(resid_cmb), 5*np.std(resid_cmb)], 
         bins=100, alpha=0.5, color='r', label='CMB: RMS={:4.2g}'.format(np.std(resid_cmb)), normed=1)
    resid_dust = dustrec[i,pixok]-maps_dust_conv[i,pixok]
    hist(resid_dust, range=[-5*np.std(resid_cmb), 5*np.std(resid_cmb)], 
         bins=100, alpha=0.5, color='b', label='Dust: RMS={:4.2g}'.format(np.std(resid_dust)), normed=1)
    title('Residuals Stokes {}'.format(stk[i]))
    legend()
tight_layout()

It works great !!!

Let's check alm space version

In [ ]:
### FGB Configuration
instrument = fgb.get_instrument('Qubic')
instrument.frequency = freqs
instrument.fwhm = np.zeros(len(freqs)) + myFWHM
components = [fgb.Dust(150., temp=20.), fgb.CMB()]

r = separate(components, instrument, get_alm_maps(maps_conv, np.zeros(len(freqs))+myFWHM, ref_arcmin=myFWHM, 
                                                  resol_correction=False, verbose=True))
print('Resulting beta: {}'.format(r.x[0]))

### Resulting CMB and Dust
almdustrec = r.s[0,:,:]
almcmbrec = r.s[1,:,:]

dustrec = hp.alm2map(almdustrec[..., ::2] + almdustrec[..., 1::2]*1j, nside)
cmbrec = hp.alm2map(almcmbrec[..., ::2] + almcmbrec[..., 1::2]*1j, nside)
dustrec[:,~pixok] = hp.UNSEEN
cmbrec[:,~pixok] = hp.UNSEEN


In [ ]:
### Resulting CMB and Dust
rc('figure', figsize=(16, 12))
suptitle('Reconstructed dust (alm based): Same Resolution', fontsize=30, y=1.05)
for i in range(3):
    hp.gnomview(dustrec[i,:], sub=(2,3,i+1), title='Dust Rec: {}'.format(stk[i]), 
                min=-3*np.std(dustrec[i,pixok]), max=3*np.std(dustrec[i,pixok]),
               rot=center, reso=15)
    hp.gnomview(dustrec[i,:]-maps_dust_conv[i,:], sub=(2,3,3+i+1), 
                title='Dust Residuals: {0:} \n RMS={1:4.2g}'.format(stk[i], 
                                                                    np.std(dustrec[i,pixok]-maps_dust_conv[i,pixok])), 
                min=-3*np.std(dustrec[i,pixok]), max=3*np.std(dustrec[i,pixok]),
               rot=center, reso=15)
tight_layout()

In [ ]:
rc('figure', figsize=(16, 12))
suptitle('Reconstructed CMB (alm based): Same Resolution', fontsize=30, y=1.05)
for i in range(3):
    hp.gnomview(cmbrec[i,:], sub=(2,3,i+1), title='CMB Rec: {}'.format(stk[i]), 
               min=-3*np.std(cmbrec[i,pixok]), max=3*np.std(cmbrec[i,pixok]),
               rot=center, reso=15)
    hp.gnomview(cmbrec[i,:]-maps_cmb_conv[i,:], sub=(2,3,3+i+1), 
                title='CMB Residuals: {0:} \n RMS={1:4.2g}'.format(stk[i], np.std(cmbrec[i,pixok]-maps_cmb_conv[i,pixok])), 
               min=-3*np.std(cmbrec[i,pixok]), max=3*np.std(cmbrec[i,pixok]),
               rot=center, reso=15)



In [ ]:
suptitle('Residuals (alm based): Same Resolution', fontsize=30, y=1.05)
for i in range(3):
    subplot(3,1,i+1)
    resid_cmb = cmbrec[i,pixok]-maps_cmb_conv[i,pixok]
    hist(resid_cmb, range=[-5*np.std(resid_cmb), 5*np.std(resid_cmb)], 
         bins=100, alpha=0.5, color='r', label='CMB: RMS={:4.2g}'.format(np.std(resid_cmb)), normed=1)
    resid_dust = dustrec[i,pixok]-maps_dust_conv[i,pixok]
    hist(resid_dust, range=[-5*np.std(resid_cmb), 5*np.std(resid_cmb)], 
         bins=100, alpha=0.5, color='b', label='Dust: RMS={:4.2g}'.format(np.std(resid_dust)), normed=1)
    title('Residuals Stokes {}'.format(stk[i]))
    legend()
tight_layout()

Not as good ! The alm transformation get sick from the edges...


# Cut-Sky simulations with different resolutions
this will require reconvolution to worst resolution

In [ ]:
synthbeam_peak150_fwhm = 0.39268176  # in degrees
myFWHM = synthbeam_peak150_fwhm * 150. / freqs  # Qubic beams

ref_fwhm = np.max(myFWHM)

maps_conv = np.array([hp.smoothing(m.copy(), fwhm=np.deg2rad(fw), pol=True, verbose=False) 
                      for m, fw in zip(maps,myFWHM)])
maps_cmb_conv = hp.smoothing(maps_cmb, fwhm=np.deg2rad(ref_fwhm), pol=True, verbose=False)
maps_dust_conv = hp.smoothing(maps_dust, fwhm=np.deg2rad(ref_fwhm), pol=True, verbose=False)
for j in range(3):
    maps_cmb_conv[j,~pixok] = hp.UNSEEN
    maps_dust_conv[j,~pixok] = hp.UNSEEN
    for i in range(nf):
        maps_conv[i,j,~pixok] = hp.UNSEEN        


In [ ]:
rc('figure', figsize=(16, 10))
suptitle('Cut-sky maps per freq. at QUBIC resolutions', fontsize=30, y=1.05)
nf = maps.shape[0]
nstk = maps.shape[1]
print(maps.shape)
for i in range(nf):
    for j in range(nstk):
        ss = np.std(maps_conv[0,j,pixok])
        hp.gnomview(maps_conv[i,j,:], sub=(nf,3,3*i+j+1), min = -1*ss, max=1*ss,
                   title='CMB+Dust {} nu={} GHz'.format(stk[j], freqs[i]), rot=center, reso=15)
tight_layout()

In [ ]:
rc('figure', figsize=(16, 5))
figure()
suptitle('CMB Only input', fontsize=30, y=1.1)
for j in range(nstk):
    ss = np.std(maps_conv[0,j,pixok])
    hp.gnomview(maps_cmb_conv[j,:], sub=(1,3,j+1), min = -1*ss, max=1*ss,
               title='CMB {} nu=150 GHz'.format(stk[j]), rot=center, reso=15)
tight_layout()

figure()
suptitle('Dust Only input', fontsize=30, y=1.1)
for j in range(nstk):
    ss = np.std(maps_conv[0,j,pixok])
    hp.gnomview(maps_dust_conv[j,:], sub=(1,3,j+1), min = -1*ss, max=1*ss,
               title='Dust {} nu=150 GHz'.format(stk[j]), rot=center, reso=15)
tight_layout()

In [ ]:

maps_reconv = reconvolve(maps_conv, myFWHM, ref_fwhm, verbose=True)

### FGB Configuration
instrument = fgb.get_instrument('Qubic')
instrument.frequency = freqs
instrument.fwhm = np.zeros(len(freqs)) + ref_fwhm
components = [fgb.Dust(150., temp=20.), fgb.CMB()]

r = separate(components, instrument, maps_reconv)

### Resulting CMB and Dust
dustrec = r.s[0,:,:]
cmbrec = r.s[1,:,:]
print('Resulting beta: {}'.format(r.x[0]))


In [ ]:
### Resulting CMB and Dust
rc('figure', figsize=(16, 12))
dustrec = r.s[0,:,:]
suptitle('Reconstructed dust (pixel space): QUBIC Resolutions', fontsize=30, y=1.05)
for i in range(3):
    hp.gnomview(dustrec[i,:], sub=(2,3,i+1), title='Dust Rec: {}'.format(stk[i]), 
                min=-3*np.std(dustrec[i,pixok]), max=3*np.std(dustrec[i,pixok]),
               rot=center, reso=15)
    hp.gnomview(dustrec[i,:]-maps_dust_conv[i,:], sub=(2,3,3+i+1), 
                title='Dust Residuals: {0:} \n RMS={1:4.2g}'.format(stk[i], 
                                                                    np.std(dustrec[i,pixok]-maps_dust_conv[i,pixok])), 
                min=-3*np.std(dustrec[i,pixok]), max=3*np.std(dustrec[i,pixok]),
               rot=center, reso=15)
tight_layout()

In [ ]:
rc('figure', figsize=(16, 12))
cmbrec = r.s[1,:,:]
suptitle('Reconstructed CMB (pixel space): QUBIC Resolutions', fontsize=30, y=1.05)
for i in range(3):
    hp.gnomview(cmbrec[i,:], sub=(2,3,i+1), title='CMB Rec: {}'.format(stk[i]), 
               min=-3*np.std(cmbrec[i,pixok]), max=3*np.std(cmbrec[i,pixok]),
               rot=center, reso=15)
    hp.gnomview(cmbrec[i,:]-maps_cmb_conv[i,:], sub=(2,3,3+i+1), 
                title='CMB Residuals: {0:} \n RMS={1:4.2g}'.format(stk[i], np.std(cmbrec[i,pixok]-maps_cmb_conv[i,pixok])), 
               min=-3*np.std(cmbrec[i,pixok]), max=3*np.std(cmbrec[i,pixok]),
               rot=center, reso=15)



In [ ]:
suptitle('Residuals: QUBIC Resolutions (pixels space)', fontsize=30, y=1.05)
for i in range(3):
    subplot(3,1,i+1)
    resid_cmb = cmbrec[i,pixok]-maps_cmb_conv[i,pixok]
    hist(resid_cmb, range=[-5*np.std(resid_cmb), 5*np.std(resid_cmb)], 
         bins=100, alpha=0.5, color='r', label='CMB: RMS={:4.2g}'.format(np.std(resid_cmb)), normed=1)
    resid_dust = dustrec[i,pixok]-maps_dust_conv[i,pixok]
    hist(resid_dust, range=[-5*np.std(resid_cmb), 5*np.std(resid_cmb)], 
         bins=100, alpha=0.5, color='b', label='Dust: RMS={:4.2g}'.format(np.std(resid_dust)), normed=1)
    title('Residuals Stokes {}'.format(stk[i]))
    legend()
tight_layout()

So now we have a bias. This is not surprising: near the edges, the reconvolutions results in incorrect values because it mixes know pixels with unknown information.

Let's first check that get get a similar bias when working in alm space.

In [ ]:
### Alm space version

### FGB Configuration
instrument = fgb.get_instrument('Qubic')
instrument.frequency = freqs
instrument.fwhm = np.zeros(len(freqs)) + ref_fwhm
components = [fgb.Dust(150., temp=20.), fgb.CMB()]

r = separate(components, instrument, get_alm_maps(maps_conv, myFWHM, ref_arcmin=ref_fwhm, 
                                                  resol_correction=True, verbose=True))
print('Resulting beta: {}'.format(r.x[0]))

### Resulting CMB and Dust
almdustrec = r.s[0,:,:]
almcmbrec = r.s[1,:,:]

dustrec = hp.alm2map(almdustrec[..., ::2] + almdustrec[..., 1::2]*1j, nside)
cmbrec = hp.alm2map(almcmbrec[..., ::2] + almcmbrec[..., 1::2]*1j, nside)
dustrec[:,~pixok] = hp.UNSEEN
cmbrec[:,~pixok] = hp.UNSEEN


In [ ]:
### Resulting CMB and Dust
rc('figure', figsize=(16, 12))
suptitle(r'Reconstructed dust (alm space): QUBIC Resolutions', fontsize=30, y=1.05)
for i in range(3):
    hp.gnomview(dustrec[i,:], sub=(2,3,i+1), title='Dust Rec: {}'.format(stk[i]), 
                min=-3*np.std(dustrec[i,pixok]), max=3*np.std(dustrec[i,pixok]),
               rot=center, reso=15)
    hp.gnomview(dustrec[i,:]-maps_dust_conv[i,:], sub=(2,3,3+i+1), 
                title='Dust Residuals: {0:} \n RMS={1:4.2g}'.format(stk[i], 
                                                                    np.std(dustrec[i,pixok]-maps_dust_conv[i,pixok])), 
                min=-3*np.std(dustrec[i,pixok]), max=3*np.std(dustrec[i,pixok]),
               rot=center, reso=15)
tight_layout()

In [ ]:
### Resulting CMB and Dust
rc('figure', figsize=(16, 12))
suptitle(r'Reconstructed CMB (alm space): QUBIC Resolutions', fontsize=30, y=1.05)
for i in range(3):
    hp.gnomview(cmbrec[i,:], sub=(2,3,i+1), title='CMB Rec: {}'.format(stk[i]), 
                min=-3*np.std(cmbrec[i,pixok]), max=3*np.std(cmbrec[i,pixok]),
               rot=center, reso=15)
    hp.gnomview(cmbrec[i,:]-maps_cmb_conv[i,:], sub=(2,3,3+i+1), 
                title='CMB Residuals: {0:} \n RMS={1:4.2g}'.format(stk[i], 
                                                                    np.std(cmbrec[i,pixok]-maps_cmb_conv[i,pixok])), 
                min=-3*np.std(cmbrec[i,pixok]), max=3*np.std(cmbrec[i,pixok]),
                rot=center, reso=15)
tight_layout()

In [ ]:
suptitle('Residuals: QUBIC Resolutions (alm space)', fontsize=30, y=1.05)
for i in range(3):
    subplot(3,1,i+1)
    resid_cmb = cmbrec[i,pixok]-maps_cmb_conv[i,pixok]
    hist(resid_cmb, range=[-5*np.std(resid_cmb), 5*np.std(resid_cmb)], 
         bins=100, alpha=0.5, color='r', label='CMB: RMS={:4.2g}'.format(np.std(resid_cmb)), normed=1)
    resid_dust = dustrec[i,pixok]-maps_dust_conv[i,pixok]
    hist(resid_dust, range=[-5*np.std(resid_cmb), 5*np.std(resid_cmb)], 
         bins=100, alpha=0.5, color='b', label='Dust: RMS={:4.2g}'.format(np.std(resid_dust)), normed=1)
    title('Residuals Stokes {}'.format(stk[i]))
    legend()
tight_layout()

So if we summarize the results:

| Method | CMB I RMS | CMB Q RMS | CMB U RMS| Dust I RMS | Dust Q RMS | Dust U RMS |
| --- | --- | --- | --- | --- | --- | --- |
| Same Resolution (Pixel based)| 2.4e-6 | 1.2e-7 | 9.9e-8 | 8.1e-7 | 4.6e-8 | 3.8 e-8 |
| Same Resolution (alm based)| 37 | 0.48 | 0.48 | 3.4 | 0.38 | 0.35 |
| QUBIC Resolution (Pixel based)| 8.1 | 0.75 | 0.66 | 6.2 | 0.61 | 0.55 |
| QUBIC Resolution (alm based)| 54 | 1.7 | 1.8 | 4.3 | 0.44 | 0.41 |

So it seems to me that there are two edge-related effects playing here:
- a "wall" effect at the sharp edges of the cut-sky when using the $a_{\ell m}$ approach (be it with the same angular resolution or not).
- A convolution effect related to zeros in the unobserved region that is likely rpesent with both pixel-based and $a_{\ell m}$ appraoches.

A possible way to investigate is to restrict the region after reconvolution...

# Solution #1: Restricting to a smaller fiducial region after reconvolution in order to limit the effect of edge-convolution leakage.

In [ ]:
maps_reconv = reconvolve(maps_conv, myFWHM, ref_fwhm, verbose=True)

newcut = 0.15
fidregion = (covmap > (newcut*np.max(covmap)))
mytrue_dust = maps_dust_conv.copy()
mytrue_cmb = maps_cmb_conv.copy()
for j in range(3):
    mytrue_cmb[j,~fidregion] = hp.UNSEEN
    mytrue_dust[j,~fidregion] = hp.UNSEEN
    for i in range(nf):
        maps_reconv[i,j,~fidregion] = hp.UNSEEN

mapregions = np.zeros(12*nside**2) + hp.UNSEEN
mapregions[pixok] = 1
mapregions[fidregion] = 2
hp.gnomview(mapregions, rot=center, reso=15, title='Fiducial region: {}'.format(newcut))

### FGB Configuration
instrument = fgb.get_instrument('Qubic')
instrument.frequency = freqs
instrument.fwhm = np.zeros(len(freqs)) + ref_fwhm
components = [fgb.Dust(150., temp=20.), fgb.CMB()]

r = separate(components, instrument, maps_reconv)



In [ ]:
### Resulting CMB and Dust
rc('figure', figsize=(16, 12))
dustrec = r.s[0,:,:]
suptitle('Reconstructed dust: QUBIC Resolutions (Fiducial cut {})'.format(newcut), fontsize=30, y=1.05)
for i in range(3):
    hp.gnomview(dustrec[i,:], sub=(2,3,i+1), title='Dust Rec: {}'.format(stk[i]), 
                min=-3*np.std(dustrec[i,fidregion]), max=3*np.std(dustrec[i,fidregion]),
               rot=center, reso=15)
    hp.gnomview(dustrec[i,:]-maps_dust_conv[i,:], sub=(2,3,3+i+1), 
                title='Dust Residuals: {0:} \n RMS={1:4.2g}'.format(stk[i], 
                                                                    np.std(dustrec[i,fidregion]-maps_dust_conv[i,fidregion])), 
               rot=center, reso=15)
tight_layout()

In [ ]:
rc('figure', figsize=(16, 12))
cmbrec = r.s[1,:,:]
suptitle('Reconstructed CMB: QUBIC Resolutions (Fiducial cut {})'.format(newcut), fontsize=30, y=1.05)
for i in range(3):
    hp.gnomview(cmbrec[i,:], sub=(2,3,i+1), title='CMB Rec: {}'.format(stk[i]), 
               min=-3*np.std(cmbrec[i,fidregion]), max=3*np.std(cmbrec[i,fidregion]),
               rot=center, reso=15)
    hp.gnomview(cmbrec[i,:]-maps_cmb_conv[i,:], sub=(2,3,3+i+1), 
                title='CMB Residuals: {0:} \n RMS={1:4.2g}'.format(stk[i], np.std(cmbrec[i,fidregion]-maps_cmb_conv[i,fidregion])), 
               rot=center, reso=15)



In [ ]:
suptitle('Residuals: QUBIC Resolutions (fiducial cut {})'.format(newcut), fontsize=30, y=1.05)
for i in range(3):
    subplot(3,1,i+1)
    resid_cmb = cmbrec[i,fidregion]-maps_cmb_conv[i,fidregion]
    hist(resid_cmb, range=[-5*np.std(resid_cmb), 5*np.std(resid_cmb)], 
         bins=100, alpha=0.5, color='r', label='CMB: RMS={:4.2g}'.format(np.std(resid_cmb)), normed=1)
    resid_dust = dustrec[i,fidregion]-maps_dust_conv[i,fidregion]
    hist(resid_dust, range=[-5*np.std(resid_cmb), 5*np.std(resid_cmb)], 
         bins=100, alpha=0.5, color='b', label='Dust: RMS={:4.2g}'.format(np.std(resid_dust)), normed=1)
    title('Residuals Stokes {}'.format(stk[i]))
    legend()
tight_layout()

So we see a significant improvement **in the pixel domain case**:
- without fiducial region (0.1 coverage cut), residuals have RMS of typically 7, 0.7, 0.7
- With a fiducial cut at 0.15, they drop by almost an order of magnitude: 1.5, 0.07, 0.07

Regarding $\beta$, we go from 0.86 to 1.465 

Let's loop on increased cut value in order to see the residuals as well as the reconstructed beta

In [ ]:
newcuts = np.logspace(-1,np.log10(0.2), 10) #np.linspace(0.1, 0.5,10)

# We reconvolve
maps_reconv = reconvolve(maps_conv, myFWHM, ref_fwhm, verbose=True)

# also use cheated maps: directly at the right resolution without reconvolving
cheat_maps_reconv = np.array([hp.smoothing(m.copy(), fwhm=np.deg2rad(ref_fwhm), pol=True, verbose=False) 
                      for m in maps])


allmapregions = np.zeros((len(newcuts), 12*nside**2))
allbetas = np.zeros(len(newcuts))
rms_dust = np.zeros((len(newcuts), 3))
rms_cmb = np.zeros((len(newcuts), 3))
allbetas_cheat = np.zeros(len(newcuts))
rms_dust_cheat = np.zeros((len(newcuts), 3))
rms_cmb_cheat = np.zeros((len(newcuts), 3))

for k in range(len(newcuts)):
    themaps_reconv = maps_reconv.copy()
    thecheatmaps_reconv = cheat_maps_reconv.copy()
    newcut = newcuts[k]

    fidregion = (covmap > (newcut*np.max(covmap)))
    mytrue_dust = maps_dust_conv.copy()
    mytrue_cmb = maps_cmb_conv.copy()
    for j in range(3):
        mytrue_cmb[j,~fidregion] = hp.UNSEEN
        mytrue_dust[j,~fidregion] = hp.UNSEEN
        for i in range(nf):
            themaps_reconv[i,j,~fidregion] = hp.UNSEEN
            thecheatmaps_reconv[i,j,~fidregion] = hp.UNSEEN
    #figure()
    #for i in range(3):
    #    hp.gnomview(mytrue_cmb[i,:], rot=center, reso=15, sub=(2,3,i+1))
    #    hp.gnomview(mytrue_dust[i,:], rot=center, reso=15, sub=(2,3,3+i+1))
    #show()

    mapregions = np.zeros(12*nside**2) + hp.UNSEEN
    mapregions[pixok] = 1
    mapregions[fidregion] = 2
    allmapregions[k,:] = mapregions
    
    #figure()
    #hp.gnomview(allmapregions[k,:], rot=center, reso=15, sub=(1,4,1))
    
    ### FGB Configuration
    instrument = fgb.get_instrument('Qubic')
    instrument.frequency = freqs
    instrument.fwhm = np.zeros(len(freqs)) + ref_fwhm
    components = [fgb.Dust(150., temp=20.), fgb.CMB()]

    ### With actual simualted maps
    r = separate(components, instrument, themaps_reconv)
    allbetas[k] = r.x[0]    
    ### Resulting CMB and Dust
    dustrec = r.s[0,:,:]
    cmbrec = r.s[1,:,:]
    dust_residuals = dustrec - mytrue_dust
    cmb_residuals = cmbrec - mytrue_cmb    
    #for i in range(3):
    #    hp.gnomview(cmb_residuals[i,:], rot=center, reso=15, sub=(1,4,i+2))
    #show()    
    rms_dust[k,:] = np.std(dust_residuals[:,fidregion], axis=1)
    rms_cmb[k,:] = np.std(cmb_residuals[:,fidregion], axis=1)

    ### With actual simualted maps
    r = separate(components, instrument, thecheatmaps_reconv)
    allbetas_cheat[k] = r.x[0]    
    ### Resulting CMB and Dust
    dustrec = r.s[0,:,:]
    cmbrec = r.s[1,:,:]
    dust_residuals = dustrec - mytrue_dust
    cmb_residuals = cmbrec - mytrue_cmb    
    #for i in range(3):
    #    hp.gnomview(cmb_residuals[i,:], rot=center, reso=15, sub=(1,4,i+2))
    #show()    
    rms_dust_cheat[k,:] = np.std(dust_residuals[:,fidregion], axis=1)
    rms_cmb_cheat[k,:] = np.std(cmb_residuals[:,fidregion], axis=1)


In [ ]:
subplot(2,1,1)
plot(newcuts, allbetas, 'ro-', label='Simulation')
plot(newcuts, allbetas_cheat, 'bo:', label='Cheat (conv before cut sky)')
xlabel('Newcut')
ylabel('Beta Dust')
legend()
axhline(y=1.54, ls=':', color='k')
subplot(2,2,3)
for i in range(3):
    p=plot(newcuts, rms_dust[:,i], 'o-', label='Dust {}'.format(stk[i]))
    plot(newcuts, rms_dust_cheat[:,i], 'o:', label='Cheat Dust {}'.format(stk[i]), color=p[0].get_color())
legend()
yscale('log')
xlabel('Newcut')
ylabel('RMS Residuals')
ylim(1e-8, 10)
subplot(2,2,4)
for i in range(3):
    p=plot(newcuts, rms_cmb[:,i], 'o-', label='CMB {}'.format(stk[i]))
    plot(newcuts, rms_cmb_cheat[:,i], 'o:', label='Cheat CMB {}'.format(stk[i]), color=p[0].get_color())
legend()
yscale('log')
xlabel('Newcut')
ylabel('RMS Residuals')
ylim(1e-8, 10)


So we do see a clear drop in the residuals when removing the edges, but it saturates fast, and the bias on beta remains. This means that this method for avoiding the reconvolution edge effect is not a good one.

So we need to explore other methods:
- Apodization before reconvolution
- Including Planck data information on the edges ?